# My Jupyter Notebook1 on IBM Data Science Experience   

**Dharmesh Patel**   
Data Solution and Analytics Professional   

_I am interested in data science because I have been working with data solutions and bussiness analytics functions of ITES organisations, and want to learn and enhance my data science skills._

### Below given code has three variable x,y and z, where value of  variable z is assigned based on addition of x and y., and prints the results with variable values as mathamatical expression.   

In [2]:
x <- 2
y <- 3
z <- x+y
print (paste0("Given z = x + y, here x = ",x," and y = ",y,", therefore z = x + y = ",x," + ",y," = ",x+y," "))

[1] "Given z = x + y, here x = 2 and y = 3, therefore z = x + y = 2 + 3 = 5 "


###### Assignment  - Use Markdowns  
Example 1 : Horizontal rule and bulleted list      

* Number of Days in current month = 30
* Numbere of Public Holidays in current month = 2
* Number of Weeklyoffs in current month = 8
* Numbere of working days in current month = 20   
---
Example 2 : Numbered list with Hyperlinks    

1. [Click here for Google Homepage](https://www.google.com "Google Homepage")    
2. [My LinkedIn Profile](https://www.linkedin.com/in/dharmesh-patel-46361134/ "Dharmesh Patel")   
3. This one is just simple text unlike the above one   
   
---
Example 3 : Images, tables, Code highlighting, strike through  

~~Table with Homtown pics~~       `Table with Hometown Images`

City   |Location   |Image   
:---   |:---       |:---
**Baroda** | _Palace_| <img src=https://upload.wikimedia.org/wikipedia/commons/a/a1/Laxmi_Vilas_Palace%2C_Vadodara.jpg width ="200">
**Baroda** | _Court_ | <img src=https://upload.wikimedia.org/wikipedia/commons/4/4a/NyayM.jpg width ="200">
**Logo** | _Github_ | ![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo")

---
`Thanks for review - Dpatel`